<a href="https://colab.research.google.com/github/viniasbr/chatbot-vestibulando/blob/main/colab-notebooks/ChatBot_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain langchain-community langchain-openai langchainhub

In [ ]:
#Imports

from google.colab import userdata

import os
import bs4

from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

In [ ]:
#Setting API Key and LLM
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAIKey')

model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
#Creating chat history function

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#Creating a model runner that invokes the session history

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
#Setting config variable with the relevant session id

config = {"configurable": {"session_id": "abc2"}}

In [ ]:
#Calling model with message history

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

'Hello Bob! How can I assist you today?'

In [ ]:
#Testing the message history function

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob. How can I assist you today, Bob?'

In [ ]:
#Changing to a session that has no history, and testing if the context changes.

config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, I do not have access to that information."

In [ ]:
#Changing back to see if previous context is still present

config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bob. How can I assist you today, Bob?'

In [ ]:
#New imports

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
#Creating prompt template, and creating a new model that: takes the input messages, appends it to the entire prompt (that starts with the message defined here)
#and pipes the whole thing into the model itself, provoking a response that has the necessary context.

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all questions to the best of your ability.",),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
#Simple call to the model with the prompt context

response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response.content

'Hello Bob! How can I assist you today?'

In [ ]:
#Redefining the history runner, with the chain model instead

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc5"}}

In [ ]:
#Testing the new runner

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

'Hello, Jim! How can I assist you today?'

In [ ]:
#Testing if history context is present

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Jim.'

In [ ]:
#Creating a prompt that has a variable in it. Invoking the model now requires a dictionary, with the "messages" key
#marking where the message inputs should go, and the "language" key being used to define the prompt with language specified

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
#It does respond in the specified language

response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Portuguese"}
)

response.content

'Olá, Bob! Como posso ajudar você hoje?'

In [ ]:
#Redefining the history runner, now with the necessary "input_messages_key" parameter

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [ ]:
config = {"configurable": {"session_id": "abc11"}}

In [ ]:
#History runner with language key. The input remains a dictionary

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Portuguese"},
    config=config,
)

response.content

'Olá, Todd! Como posso ajudar você hoje?'

In [ ]:
#Testing history context with language key

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Portuguese"},
    config=config,
)

response.content

'Seu nome é Todd. Como posso ajudar você hoje, Todd?'

# History Management


In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
#Function to send only last k messages to context window

def filter_messages(messages, k=10):
    return messages[-k:]

In [ ]:
#Redefining chain using message filtering

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [ ]:
#10 example messages, with the first being the name of the user. Once a new message is appended,
#the LLM should not know the users name.

messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [ ]:
#This should not be known

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I'm sorry, I don't have access to your personal information."

In [ ]:
#This should be known

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my fav ice cream")],
        "language": "English",
    }
)
response.content

'You mentioned earlier that you like vanilla ice cream.'

In [ ]:
#Implementing the new chain in the history runner

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

In [ ]:
#Now, the calls we made get added to the history, alongside with the predefined messages

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have access to personal information."

In [ ]:
#Now that we have two new messages in the history, the ice cream flavour should
#not be known

response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="whats my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, I don't have that information."